# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Селиванова Валерия

### Общая информация

__Дата выдачи:__ 06.04.2020

__Дедлайн:__ 23:59 20.04.2020


### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
min(\text{points}, 18)  \times 10 / 18,
$$

где points — количество баллов за обязательную часть, которое вы набрали. Максимальное число баллов, которое можно получить за обязательную часть — 18, за каждые полтора балла сверху вы получите 1 бонусный балл (максимум 2). Также вы можете использовать бонусные баллы, которые накопили ранее.

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-6: rd5CNrr

Перед отправкой перезагрузите ноутбук и проверьте, что все ячейки могут быть последовательно выполнены. Ноутбук должен запускаться с использованием python 3.6+

### Подготовка данных

In [39]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [40]:
# Качаем датасет

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

/bin/sh: wget: command not found


### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [41]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [42]:
data.shape

(30000, 4)

In [43]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [45]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

In [46]:
y_train[:5]

array([ 27,  20,  84, 106,  27])

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Можете использовать WordPunctTokenizer или подобрать какой-то другой, если считаете, что он лучше подойдет для этой задачи.


In [47]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'

print("before:", text,)
print("after:", tokenizer.tokenize(text.lower()),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['здраствуйте', '.', 'я', ',', 'кирилл', '.', 'хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже

In [50]:
def preprocess(text: str) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    a = tokenizer.tokenize (text.lower())
    b = ' '.join (a)
    
    return b

In [51]:
assert preprocess(text) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте title и description в train и test

In [52]:
X_train = np.array ([[preprocess (title), preprocess (description)] for title, 
                     description in X_train])
X_test = np.array ([[preprocess (title), preprocess (description)]for title, 
                    description in X_test])

In [53]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (3 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где в соответствии каждому токену стоит количество раз, которое оно встретилось в X_train

In [64]:
from collections import Counter
a = np.concatenate (X_train, axis = None)
b = map (str, np.concatenate ([i.split() for i in a], axis = None))

tokens = Counter (b)

In [65]:
assert tokens['сапоги'] == 454

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов

In [67]:
c = sorted (tokens.items(), key = lambda item: -item [1]) #частотные
print ([k for k, v in c][:10])
d = sorted (tokens.items(), key = lambda item: item [1]) #редкие
print ([k for k, v in d][:10])

['/', ',', '.', '-', 'в', 'и', 'на', './', ':', 'с']
['iqmac', 'ядерным', 'ресурсоемкие', 'быстродействием', 'кинематографическому', 'ирис', 'саженец', 'корень', 'зубчаниновка', 'боярышник']


__Задание:__ оставьте в словаре только топ 10000 самых частотных токенов

In [69]:
tokens = {k: v for k, v in c [:10000]}
tokens1 = list (tokens.keys())

__Задание:__ реализуйте функцию, которая предложение переводит в вектор из чисел. То есть каждому слову из словаря сопоставляется количество раз, которое оно встретилось в предложении.

In [80]:
def text_to_bow(text: str, tokens1: list) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    input: строка
    output: вектор размерности словаря
    """
    
    a = text.split()
    b = dict (Counter (a))
    c = np.array ([b.get (i, 0) for i in tokens1])
    
    return c

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого текста из description сопоставляет вектор.

In [81]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    
    des = items [:, 1]
    a = np.array ([text_to_bow (i, tokens1) for i in des])
    
    return a

In [82]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [83]:
# чтобы видеть проход по итерациям можно использовать библиотеку tqdm
# она работает примерно так
from tqdm import tqdm

for i in tqdm(range(100)):
    pass

100%|██████████| 100/100 [00:00<00:00, 13966.12it/s]


In [84]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [88]:
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
X_train_bow1 = csr_matrix (X_train_bow)
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_train_bow1, y_train)
ypr = lr_model.predict (X_test_bow)
print('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

assert accuracy_score(lr_model.predict(X_test_bow), y_test) > 0.7

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.7046666666666667


In [91]:
from sklearn.svm import LinearSVC
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_train_bow1, y_train)
ypr = svc_model.predict (X_test_bow)
print('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

assert accuracy_score(svc_model.predict(X_test_bow), y_test) > 0.68

Accuracy: 0.6842222222222222


### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [93]:
def title (items: np.array, tokens1:list) -> np.array:
    a = items [:, 0]
    b = np.array ([text_to_bow (i, tokens1) for i in a])
    return b
X_train_bow2 = X_train_bow + title (X_train, tokens1)
X_test_bow2 = X_test_bow + title (X_test, tokens1)
X_train_bow3 = csr_matrix (X_train_bow2)

In [96]:
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_train_bow3, y_train)
ypr = lr_model.predict (X_test_bow2)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.7837777777777778


In [95]:
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_train_bow3, y_train)
ypr = svc_model.predict (X_test_bow2)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.7535555555555555


__Для обоих моделей качество повысилось__

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [215]:
import sklearn
X_train_bow4 = sklearn.preprocessing.normalize (X_train_bow1)
X_test_bow3 = sklearn.preprocessing.normalize (X_test_bow)

In [212]:
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_train_bow4, y_train)
ypr = lr_model.predict (X_test_bow3)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.5833333333333334


In [213]:
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_train_bow4, y_train)
ypr = svc_model.predict (X_test_bow3)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

Accuracy: 0.7244444444444444


Качество для обеих моделей ухудшилось. Это связано с тем, что нормализация существенно не меняет данные. Не изменяются веса отдельных слов, расстояния между текстами и т. д., поэтому качество модели и не улучшается.

### Mystem (1 балл)

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [105]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

/bin/sh: wget: command not found
tar: Error opening archive: Failed to open 'mystem-3.0-linux3.1-64bit.tar.gz'
cp: mystem: No such file or directory


In [106]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /private/var/folders/td/jfnkrcjn0h94yj1jkppgbvzh0000gn/T/pip-req-build-9q2v09ns
  Running command git clone -q https://github.com/nlpub/pymystem3 /private/var/folders/td/jfnkrcjn0h94yj1jkppgbvzh0000gn/T/pip-req-build-9q2v09ns
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp37-none-any.whl size=9922 sha256=1ced844cb818963df1b3876cdde1cb3f651cc264d20b8a6f99b70c74193721cf
  Stored in directory: /private/var/folders/td/jfnkrcjn0h94yj1jkppgbvzh0000gn/T/pip-ephem-wheel-cache-p8t3bgce/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3
Note: you may need to restart the kernel to use updated packages.


In [107]:
from pymystem3 import Mystem
mystem = Mystem (entire_input = False)
X_trainl = [[mystem.lemmatize (t), mystem.lemmatize (d)] for t, d in X_train]
X_testl = [[mystem.lemmatize (t), mystem.lemmatize (d)] for t, d in X_test]
X_trainl = [' '.join (np.concatenate (z)) for z in X_trainl]
X_testl = [' '.join (np.concatenate (z)) for z in X_testl]

Installing mystem to /Users/valeria/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


In [108]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
thenword = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/valeria/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [110]:
from sklearn.feature_extraction.text import CountVectorizer
N = CountVectorizer (stop_words = thenword)
X_trainl1 = N.fit_transform (X_trainl)
X_testl1 = N.transform (X_testl)

In [111]:
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_trainl1, y_train)
ypr = lr_model.predict (X_testl1)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.7972222222222223


In [112]:
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_trainl1, y_train)
ypr = svc_model.predict (X_testl1)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

Accuracy: 0.7841111111111111


Качество для обоих моделей увеличилось.

### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для title и description (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация title и description).

__Задание:__ составьте словарь, где каждому слову из изначального словаря будет стоять в соответствии количество документов, где это слово встретилось.

In [113]:
from tqdm.notebook import tqdm

In [114]:
doc = map (' '.join, X_train)
w = [i.split() for i in doc]
def dcnt (word: str):
    a = sum ([word in b for b in w])
    return a
dic = {word : dcnt (word) for word in tqdm (tokens1)}

__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. 

In [129]:
n = len (X_train)
def text_to_tfidf(text: str, dic: dict, tokens1: list, n: int) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указан tf-idf
    """
    
    a = text_to_bow (text, tokens1)
    b = len (text.split())
    c = [(a [i] / b) * np.log (n / dic [word]) for i, word in enumerate (tokens1)]
    d = np.array (c)
    return d

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf.

In [130]:
def items_to_tfidf(items: np.array) -> np.array:
    """ 
    Для каждого товара возвращает его tfidf вектор
    """
    a = [text_to_tfidf (t + ' ' + d, dic, tokens1, n) for t, d in tqdm (items)]
    b = np.array (a)
    return b

In [131]:
X_train_tfidf = items_to_tfidf(X_train)
X_test_tfidf = items_to_tfidf(X_test)

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [132]:
X_train_tfidf1 = csr_matrix (X_train_tfidf)
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_train_tfidf1, y_train)
ypr = lr_model.predict (X_test_tfidf)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

Accuracy: 0.6628888888888889


In [133]:
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_train_tfidf1, y_train)
ypr = svc_model.predict (X_test_tfidf)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

Accuracy: 0.7942222222222223


Качество логистической регрессии снизилось, а SVC модели увеличилось.

### Hashing Vectorizer (1 балл)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [161]:
from sklearn.feature_extraction.text import HashingVectorizer
import nltk
nltk.download ('punkt')

[nltk_data] Downloading package punkt to /Users/valeria/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [162]:
vectorizer = HashingVectorizer (tokenizer = nltk.tokenize.word_tokenize, analyzer = 'word')
X_trainv = vectorizer.fit_transform ([i [0] + ' ' + i [1] for i in X_train])
X_testv = vectorizer.fit_transform ([i [0] + ' ' + i [1] for i in X_test])

In [163]:
X_trainv1 = csr_matrix (X_trainv)
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_trainv1, y_train)
ypr = lr_model.predict (X_testv)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.6503333333333333


In [164]:
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_trainv1, y_train)
ypr = svc_model.predict (X_testv)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

Accuracy: 0.8106666666666666


### Word Vectors (3 балла)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [168]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

/bin/sh: wget: command not found


In [173]:
!tar -xzf ru.tar.gz

In [174]:
!pip install gensim

In [175]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

In [176]:
# как мы видим, каждому словву данная модель сопоставляет вектор размерности 300

print(model['привет'].shape)
print(model['привет'])

(300,)
[ 0.02916384  0.02167605  0.05127367 -0.00971958  0.0465235  -0.03945766
  0.02737866  0.00638128 -0.03774629 -0.04257201 -0.00995653  0.02291315
 -0.02301722  0.06697998 -0.03674482 -0.02403202 -0.05404469  0.01372932
  0.00926399 -0.0013149   0.11941359 -0.022448    0.04011497  0.06980549
  0.00407011 -0.09384539  0.03050164 -0.02578281 -0.03525181 -0.06603175
  0.04752798  0.05874675  0.01983666  0.06092105 -0.00957561  0.08307806
 -0.01288903  0.04705157  0.02198839 -0.00649013 -0.0171444   0.03302203
  0.02124882 -0.01902875 -0.05235172  0.03458685 -0.01409259 -0.07477519
  0.01916078  0.02985001  0.0086322   0.03051201  0.02831862  0.04549561
  0.00761138 -0.05459622  0.09056009 -0.08807947 -0.05420396 -0.04793203
 -0.05672329 -0.03025264 -0.03024072 -0.05890108 -0.03137474  0.03292617
  0.05440779 -0.04548327 -0.07266086 -0.09327219  0.07247883  0.0111061
  0.01824225 -0.10570452  0.05110046 -0.04659343 -0.03277056 -0.00803401
 -0.03978698  0.00826598 -0.01074128  0.01843

In [204]:
# Будем рассматривать эмбеддинг предложения, как сумму эмбеддингов токенов

def sentence_embedding(sentence: str, model) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    
    a = sentence.split ()
    b = np.array ([model [i] for i in a if i in model.wv])
    c = np.sum (b, axis = 0)
    if c.shape == (300, ):
        return c
    else:
        return np.zeros ((300, ))

In [205]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', model)[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

__Задание:__ сделайте все то же, что в предыдущих пунктах -- реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество.

In [206]:
def embd (items: np.array, model) -> np.array:
    a = [sentence_embedding (t + ' ' + d, model) for t, d in tqdm (items)]
    b = np.array (a)
    return a

In [207]:
X_trainem = embd (X_train, model)
X_testem = embd (X_test, model)

In [208]:
X_trainem1 = csr_matrix (X_trainem)
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_trainem1, y_train)
ypr = lr_model.predict (X_testem)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.5781111111111111


In [209]:
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_trainem1, y_train)
ypr = svc_model.predict (X_testem)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.5712222222222222


### Что дальше? (6 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать N-Gram модели текстовой классификации (__1.5 балла__)

2. Поработать с другими эмбеддингами для слов (например word2vec или GloVe) (__1.5 балла__)

3. Другие способы токенизации (pymorphy2, spaCy) (__1.5 балла__)

4. Добиться качества > 0.765 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__1.5 балла__)

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

__Пункт 1__

Сделаем `N-Gram` модели через `SciKitLearn`

In [241]:
ngram = CountVectorizer (stop_words = thenword, ngram_range = (1, 5))
X_trainn = ngram.fit_transform (X_trainl)
X_testn = ngram.transform (X_testl)

In [242]:
X_trainem1 = csr_matrix (X_trainem)
lr_model = LogisticRegression (random_state = 42, max_iter = 500)
lr_model.fit (X_trainn, y_train)
ypr = lr_model.predict (X_testn)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valeria/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.7718888888888888


In [243]:
svc_model = LinearSVC (random_state = 42, max_iter = 10000)
svc_model.fit (X_trainn, y_train)
ypr = svc_model.predict (X_testn)
print ('Accuracy: {}'.format (accuracy_score (y_test, ypr)))

Accuracy: 0.7764444444444445


Качество моделей изменилось немного снизилось по сравнению с использованием униграмм в майстеме (было 0.797 для логистической регрессии и 0.784 для опорных векторов). Это может быть связано с тем, что для наших целей на наших данных важнее отдельные слова, а не их сочетания, т. к. поиск товаров обычно осуществляется как раз по ключевым словам.

__Пункт 4__

Качество > 0.765 уже было достигнуто при использовании некоторых методов ранее.

При использовании `MyStem` (0.797 для логистической регрессии и 0.784 для SVC), `хэшинга` (0.81 для опорных векторов) и `n-gram` модели.